In [1]:
from bs4 import Tag, NavigableString, BeautifulSoup
import requests

First parent/root soup

https://old.reddit.com/r/MechanicalKeyboards/comments/k385xo/mech_mat_giveaway/

Individual comment with replies and replies within replies soup

https://old.reddit.com/r/MechanicalKeyboards/comments/k385xo/mech_mat_giveaway/ge0pz8h/

In [2]:
def getRedditSoup(url):
    headers = {
        "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
        "accept-encoding": "gzip, deflate, br",
        "accept-language": "en-US, en; q=0.9",
        "sec-fetch-dest": "document",
        "sec-fetch-mode": "navigate",
        "sec-fetch-site": "same-origin",
        "sec-fetch-user": "?1",
        "upgrade-insecure-requests": "1",
        "user-agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.193 Safari/537.36"
    }
    response = requests.get(url, headers = headers)

    if (response.status_code != 200):
        return BeautifulSoup("", "html.parser")

    return BeautifulSoup(response.text, "html.parser")

In [49]:
def extractCommentData(comment_tag):
    top_level_comment_object = {}
    score = comment_tag.find("span", class_="score unvoted")["title"]
    author = comment_tag.find("a", class_="author").get_text(strip=True).strip()
    
    date_posted = comment_tag.find("time", class_="live-timestamp")
    date_posted_timestamp = date_posted["datetime"]
    date_posted_readable = date_posted["title"]
    
    date_edited = comment_tag.find("time", class_="edited-timestamp")
    date_edited_timestamp = date_edited["datetime"]\
        if date_edited != None else None
    
    num_children = comment_tag.find("a", class_="numchildren")\
        .text.strip().replace("(", "").replace(")", "").split(" ")[0]
    
    permalink = comment_tag.find("a", class_="bylink")
    
    comment_container = comment_tag\
        .find("div", class_="usertext-body may-blank-within md-container")\
        .find("div", class_="md")
    
    comment_formatted = comment_container.prettify()
    comment_raw = "".join([p.text for p in comment_container.find_all("p")]).strip().rstrip()
    
    top_level_comment_object["score"] = score
    top_level_comment_object["author"] = author
#     top_level_comment_object["date_posted_timestamp"] = date_posted_timestamp
#     top_level_comment_object["date_posted_readable"] = date_posted_readable
#     top_level_comment_object["date_edited"] = date_edited
#     top_level_comment_object["num_children"] = num_children
#     top_level_comment_object["permalink"] = permalink
#     top_level_comment_object["comment_formatted"] = comment_formatted
    top_level_comment_object["comment_raw"] = comment_raw
    
    return top_level_comment_object
    
#     if (num_children == 0):
#         base case
#     else:
#         nested_soup
        

In [4]:
with open('../../samples/reddit_post_with_nested_comments.txt', 'r') as file:
    data = file.read().replace('\n', '')
    
soup = BeautifulSoup(data, "html.parser")

In [14]:
nested_comments_container = soup.find_all("div", class_=["nestedlisting"])[0]
# top_level_comments = nested_comments_container.find_all("div", class_="comment", recursive=False)

first = nested_comments_container.find("div", class_="comment")

top_level_comments_objects = []

top_level_comments_objects.append(extractCommentData(first))
top_level_comments_objects.extend([extractCommentData(element) for element in first.next_siblings 
                      if (isinstance(element, Tag) and "comment" in element["class"])])

In [57]:
# top_level_comment_objects

Extracting comments from a comment with replies (after https://old.reddit.com/r/MechanicalKeyboards/comments/k385xo/mech_mat_giveaway/ge0pz8h/ request)

In [16]:
with open('../../samples/reddit_comment_with_replies.txt', 'r') as file:
    inside_data = file.read().replace('\n', '')
    
nested_soup = BeautifulSoup(inside_data, "html.parser")

In [65]:
container = nested_soup.find_all("div", class_=["nestedlisting"])[0]
container_comments = container.find_all("div", class_=["noncollapsed", "comment"])
first_reply = container_comments[1] 
# We want to get comment at index 1 since it is the first reply to our parent comment and then get its 
# siblings(same-level)

child_comment_objects = []
child_comment_objects.append(extractCommentData(first_reply))
child_comment_objects.extend([extractCommentData(sibling) for sibling in first_reply.next_siblings
                             if (isinstance(sibling, Tag) and "comment" in sibling["class"])])

In [66]:
# For some reason there's trailing \\n characters in the strings -_-, have to get rid of that.

In [67]:
parent_comment = extractCommentData(container_comments[0]) 

In [59]:
child_comment_objects

[{'score': '116',
  'author': '\\n            tenchi4u\\n',
  'comment_raw': '\\n              Nice.  Now do Evangelion 🔥😜👍\\n'},
 {'score': '3',
  'author': '\\n            fiction99\\n',
  'comment_raw': '\\n              Looking good!\\n'},
 {'score': '3',
  'author': '\\n            MrTsunami4\\n',
  'comment_raw': '\\n              y\\n'},
 {'score': '1',
  'author': '\\n            Humbletypes\\n',
  'comment_raw': '\\n              Wowowow thats nice 😍\\n'},
 {'score': '1',
  'author': '\\n            VictorVonDoom99\\n',
  'comment_raw': '\\n              This would be super helpful with my new desk coming In\\n'},
 {'score': '1',
  'author': '\\n            olphinator\\n',
  'comment_raw': '\\n              Looking sick!\\n'},
 {'score': '1',
  'author': '\\n            Myth506\\n',
  'comment_raw': '\\n              That looks so nice!\\n'},
 {'score': '1',
  'author': '\\n            Raion1\\n',
  'comment_raw': '\\n              Looks cool\\n'},
 {'score': '1',
  'author': 

In [68]:
parent_comment["replies"] = child_comment_objects

In [69]:
parent_comment

{'score': '333',
 'author': '\\n         IllustriousPhysics\\n',
 'comment_raw': "\\n           Inspired by early 2000\\'s Gundams.\\n          \\n           \\u200b\\n          \\n           We love to bring high resolution art onto deskmats. The Mech Mat is no different, but for this piece, we knew we needed something different to capture the details of the main design. The illustration is drawn using vectors, which are based on mathematical formulas. This ensures the original high level of detail, regardless of how large the print area is.\\n          \\n           Renders :\\n           \\n            https://imgur.com/a/tks3kyV\\n           \\n          \\n           \\u200b\\n          \\n           \\n            We are honestly speechless seeing the comments and support by the community. In spirit of Cyber Monday, we are offering a 20% discount at launch for 24 hours.  This discount will be available for both the Mech Mat and Fujin Raijin Deskmat!\\n           \\n          \\n 